# Models for Surgery and Outcomes

In this workbook, I apply the Lasso to developing parsimonious models for surgical outcomes and for application of treatment. The ideas follows the basic recipes in [Farrell (2015)](https://arxiv.org/abs/1309.4686). As I understand it, one implementation of the procedure is:

1. Start with a perhaps large set of plausible variables, and fit a model of treatment/surgery selection with the Lasso to pare down the number of variables predicting treatment.
2. Using a group lasso, estimate models of outcomes. Grouping the coefficients across the outcome models ensures that a uniform set of outcome predictors will be used. 
3. With these models, estimate treatment effects using a doubly-robust estimator, such as the augmented-inverse-probability-weighted estimator. This involves:
    - Create propensity scores using the treatment model developed in step 1.
    - Form weights using the propensity scores. Estimate outcome models and form treatment effects from outcome model predictions. 
    - Use weighted averages to compute the average treatment effect (ATE).

First: set up the python `ipystata` package, and changing the directory to the right place. I begin by describing the data. 

In [11]:
import ipystata
import os

cd = os.getcwd()
cdl = [cd]

In [12]:
%%stata -s gl -i cdl
chdir "`cdl'"

clear all 
set more off 

use "`cdl'\DataFiles\WorkingData.dta"

describe


C:\Users\Matthew\Documents\github\CivilWarSurgery

Contains data from C:\Users\Matthew\Documents\github\CivilWarSurgery\DataFiles\WorkingData.dta
  obs:           498                          
 vars:            43                          6 Oct 2017 12:09
 size:       224,598                          
------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
------------------------------------------------------------------------------------------------------------------------------------------
type            str20   %20s                  Type
case            int     %10.0g                Case
name            str11   %11s                  Name
regno           str3    %9s                   RegNo
regstate        str18   %18s                  RegState
injury          str99   %99s                  Injury
ope

The following is a set of commands that create globals to make passing variables easier, standardizes continuous variables, and then calculates the maximum value across all the variables for use in the lasso. 

In [13]:
%%stata -s gl

global contvars sev sev2 longname lncas
global contvarsex sev sev2 
global dumvars td1 td2 td3 td4 td5 td6 td7 td8 td9 td10
global dumregs Ky22 Oh16 In54 Oh42 Il13 Mo29 In49 Mo06 

global yvar operated

foreach v of global contvars {
    quietly sum `v'
    replace `v' = (`v' - r(mean))/r(sd)
}

global xlist $contvars $dumvars $dumregs


(498 real changes made)
(498 real changes made)
(498 real changes made)
(498 real changes made)



Now, we have to standardize all the non-dummy variables, calculate the maximum value and also calculate $N$. This is useful for the recipe in [Farrell (2015)](https://arxiv.org/abs/1309.4686). 

## The program

To do the analysis, I follow the previously-mocked-up Lasso program discussed [in this workbook](Developing%20Group%20Lasso%20Programs%20In%20Stata.ipynb). Here it is:

In [14]:
%%stata -s gl
mata:
    void gll_mata(M, todo, b, f, gd, H) {
        
        y  = moptimize_util_depvar(M, 1)
        xb = moptimize_util_xb(M, b, 1)
        
        lam = moptimize_util_userinfo(M, 1)
        g   = moptimize_util_userinfo(M, 2)
        w   = moptimize_util_userinfo(M, 3)
        w   = w/colsum(w)*rows(w)
        
        bt  = b[1::cols(b) - 1]
        gb  = sqrt(rowsum(g)):*sqrt(rowsum((bt:^2):*g))
        norm = colsum(gb)       
       
        lnf  = w:*((y :== 1) :*xb - ln(1 :+ exp(xb))) 
        
        f = colsum(lnf) - norm*lam      
    }
end   


note: argument todo unused
note: argument gd unused
note: argument H unused



For the first Lasso, I first fit a completely unrestricted model - this is equivalent to using an identity grouping matrix, so we are not actually using a grouped lasso, but a special case of a group lasso, in which each variable is its own group. This makes computing the grouping matrix and lambda relatively easy.

In [15]:
%%stata -s gl
mata:
    st_view(X=.,.,"$xlist")
    st_view(y=.,.,"$yvar")
    
    w = J(rows(y),1,1)
    
    lambda = 2*max(abs(X))/sqrt(rows(X))*(1+ln(rows(X))^(3/2+1))^(1/2)
    printf("Lambda is")
    lambda
    g = I(cols(X))
    printf("Number variables")
    cols(X)
end


Lambda is
  2.711880252

Number variables
  22



## Running the Lasso on Treatment

We now run the Lasso on the treatment variable. **Caution:** this can take awhile!

In [16]:
%%stata -s gl

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gll_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_trace_value(M, "off")
    moptimize_init_conv_maxiter(M, 2000)
    moptimize(M)
    moptimize_result_display(M)
end 


                                                Number of obs     =        498

------------------------------------------------------------------------------
           y |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         sev |   .9031894   .1195488     7.55   0.000      .668878    1.137501
        sev2 |   1.41e-07   .0009665     0.00   1.000    -.0018942    .0018945
    longname |  -.0941415   .1145347    -0.82   0.411    -.3186254    .1303425
       lncas |  -.3851187   .1132135    -3.40   0.001     -.607013   -.1632244
         td1 |   .5871956   .4273241     1.37   0.169    -.2503442    1.424735
         td2 |   1.45e-06   .0010378     0.00   0.999    -.0020326    .0020355
         td3 |   2.22e-06   .0014392     0.00   0.999    -.0028186    .0028231
         td4 |  -.0000232   .0047313    -0.00   0.996    -.0092964    .0092501
         td5 |   1.187457   .3733153     3.18   0.

The Lasso selects the following variables: `sev`, `longname`, `lncas`, `td1`, `td5`, `td6`, `td8`, `td9`, `td10`, `Mo29`,  and `Mo06`. So, severity of injury, having a longname/middle initial, a large amount of regimental casualties matter for treatment. `td1`, `td5`, `td6`, and `td8` are positive predictors of surgery - this makes sense, as these denote arm, hand, and knee injuries. `td9` and `td10` seem to be a strong negative predictors, but the interaction terms for these variables go in the opposite direction. Membership in regiments perhaps can be motivated by time and place on the battlefield. 

For the Lasso report, we note that the lasso reduced the model from 22 variables to 11.

## The reduced model

Here is the refit, parsimonious model selected by the Lasso.

In [17]:
%%stata -s gl

logit operated sev longname lncas td1 td5 td6 td8 td10 Mo29  Mo06, robust
predict ps, p
gen w = 1/ps*operated + 1/(1-ps)*(1-operated)
drop ps


Iteration 0:   log pseudolikelihood = -303.87074  
Iteration 1:   log pseudolikelihood = -230.93525  
Iteration 2:   log pseudolikelihood = -227.52892  
Iteration 3:   log pseudolikelihood = -227.47665  
Iteration 4:   log pseudolikelihood = -227.47651  
Iteration 5:   log pseudolikelihood = -227.47651  

Logistic regression                             Number of obs     =        498
                                                Wald chi2(10)     =      99.18
                                                Prob > chi2       =     0.0000
Log pseudolikelihood = -227.47651               Pseudo R2         =     0.2514

------------------------------------------------------------------------------
             |               Robust
    operated |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         sev |    .939183   .1275482     7.36   0.000     .6891931    1.189173
    longname |  -.16236

## (Group) Lassoing of Outcomes

I now fit the grouped lasso to get some parsimonious models of outcomes. The chief idea is to generate interaction terms for outcomes, and then group each coefficient with its interaction term. 

# Outcome control variables:

To select control variables for the outcome equation, I do the following. I first interact each variable with the operated dummy, so that we now have, effectively, two separate sets of coefficients: those that predict the outcome for the "operated" sample, and those that predict the outcome for the "not operated" sample. 

The coefficients and interacted coefficients are then grouped, so only those variables that help in predicting outcomes among both operated and not operated groups get included as additional controls.

Note that since the variables are again dummies, and the rest of the candidate explanatory variables have already been normalized, we do not need to go through that step again. 

### Generating interactions

Here, we remake our list of variables, omitting our instruments `longname` and `lncas`. But note that it actually doesn't matter if one leaves them in as they get dropped by the lasso.

In [18]:
%%stata -s gl

global xlist $contvarsex $dumvars $dumregs 

global xoutlist
foreach variable in $xlist {
    gen io_`variable' = `variable'*operated
    global xoutlist $xoutlist `variable' io_`variable'
}

global yvar outcome

A trick for making the grouping matrix is then to just double up an identity matrix. This means that the coefficient on a variable and its interaction term is grouped together. So, if one is included, both are - meaning that the variable has to have explanatory power in predicting the outcome for both the treated/surgical and non-treated/nonsurgical group. 

**Notes**: the following two blocks of code can take awhile to run. The optimization routine runs in two stages. In the first stage, it uses Nelder-Mead which is fast but crude. Once it gets close to the optimum, it switches to the conventional maximization routine, which is more able to set coefficients close to zero.

In [19]:
%%stata -s gl 
mata:
    st_view(X=.,.,"$xoutlist")
    st_view(y=.,.,"$yvar")
    st_view(w=.,.,"w")
    
    lambda = 2*max(abs(X))/sqrt(rows(X))*(1+ln(rows(X))^(3/2+1))^(1/2)

    g = I(cols(X)/2)#J(1,2,1)
end

In [20]:
%%stata -s gl -os

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gll_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_conv_maxiter(M, 50000)
    moptimize_init_trace_value(M, "off")
    moptimize_init_technique(M, "nm")
    moptimize_init_nmsimplexdeltas(M,J(1,cols(X),.05))
    moptimize(M)
    b = moptimize_result_coefs(M)
end  

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gll_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_conv_maxiter(M, 1000)
    moptimize_init_trace_value(M, "off") 
    moptimize_init_eq_coefs(M, 1, b)    
    moptimize(M)
    moptimize_result_display(M)
end


                                                Number of obs     =        498

------------------------------------------------------------------------------
           y |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         sev |   -.568967   .1400847    -4.06   0.000     -.843528   -.2944061
      io_sev |   .1883502   .2014239     0.94   0.350    -.2064334    .5831337
        sev2 |   5.80e-07   .0010137     0.00   1.000    -.0019862    .0019873
     io_sev2 |   1.48e-06   .0009563     0.00   0.999    -.0018729    .0018758
         td1 |   .4454108   .4347585     1.02   0.306    -.4067003    1.297522
      io_td1 |   .0947225   .3148721     0.30   0.764    -.5224155    .7118605
         td2 |   8.17e-07   .0008692     0.00   0.999    -.0017027    .0017043
      io_td2 |   8.46e-07    .000872     0.00   0.999    -.0017083      .00171
         td3 |  -3.34e-07   .0008634    -0.00   1.

Glancing at the above coefficients, we see that the real contributors are: `sev`, `td1`, `td5`, `td8`, and `td10`, `Oh16`, and `Il13`. One could quibble about a few other variables, but these very seldomly make an impact on what follows.  

For the lasso, we start with 20 variables grouped in sets of 2. The right way of saying things is that our initial set of 20 groups of variables is reduced to 5 groups of variables.  

**Note**: while the routine above says that it has not converged, this s really just because the coefficients that are effectively zero are being pushed further and further towards zero.

# Estimating treatment effects

Now that I have trimmed the model down to its components, I follow  [Farrell (2015)](https://arxiv.org/abs/1309.4686) and use the parisominious model to estimate treatment effects. Here, I just preview results using `Stata`'s canned `teffects` package. But in the workbook where I actually put together the treatment effects and graphs, I do it by hand following [Wooldridge, Chapter 21, pp. 930-931](https://mitpress.mit.edu/books/econometric-analysis-cross-section-and-panel-data).

Reason being, I'm not sure exactly what Stata's package is doing, and there doesn't appear to be any documentation describing how it works in detail. But as a preview, it works fine:

In [23]:
%%stata -s gl -os

teffects aipw (outcome sev td1 td5 td8 td10 Il13, logit) ///
      (operated sev longname lncas td1 td5 td6 td8 td10 Mo29 Mo06, logit), aequations

    logit outcome sev td1 td5 td8 td10 Il13 [pweight=w] if operated == 1


Iteration 0:   EE criterion =  1.403e-15  
Iteration 1:   EE criterion =  2.267e-27  

Treatment-effects estimation                    Number of obs     =        498
Estimator      : augmented IPW
Outcome model  : logit by ML
Treatment model: logit
------------------------------------------------------------------------------
             |               Robust
     outcome |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
ATE          |
    operated |
   (1 vs 0)  |    .113198    .039278     2.88   0.004     .0362145    .1901815
-------------+----------------------------------------------------------------
POmean       |
    operated |
          0  |   .7657844   .0271102    28.25   0.000     .7126494    .8189194
-------------+----------------------------------------------------------------
OME0         |
         sev |  -.9910786   .1697128    -5.84   0.000     -1.32371   -.6584476
       

# An extension - three treatments:

## Resection/Excision, Amputation, and Conservation

Here, following up a referee's suggestion, I look at amputation, versus excision/resection, versus no treatment, which is the typical breakdown followed in the Compendium of the War of the Rebellion. The logic here is that there is a difference between amputation and excision, as the latter involves digging around in a wound. Is there a difference in recovery rates when looking at these things? 

While this seems like an obvious thing to do, it wasn't obvious how to estimate outcome models that made sense. Of course, the lasso provides a means of picking out a parsimonious outcome model when there are multiple treatments. 

The very first thing that is needed is a lasso function that can handle multiple alternatives. Here is the `Mata` function:

In [24]:
%%stata -s gl

mata:
    void gltril_mata(M, todo, b, f, gr, H) {
        
        y  = moptimize_util_depvar(M, 1)
        xb1 = moptimize_util_xb(M, b, 1)
        xb2 = moptimize_util_xb(M, b, 2)

        lam = moptimize_util_userinfo(M, 1)
        g   = moptimize_util_userinfo(M, 2)
        w   = moptimize_util_userinfo(M, 3)
        w   = w/colsum(w)*rows(w)
        
        b1  = b[1::cols(b)/2]
        b2  = b[cols(b)/2 + 1::cols(b)]

        b1 = b1[1::cols(b1)-1]
        b2 = b2[1::cols(b2)-1]

        bt = b1, b2

        gb  = sqrt(rowsum(g)):*sqrt(rowsum((bt:^2):*g))
        norm = colsum(gb)       
    
        lnf  = (y :== 1) :*xb1 + (y :==2 ) :*xb2 - ln(1 :+ exp(xb1) :+ exp(xb2))       
        
        f = sum(lnf) - norm*lam
    }
end   


note: argument todo unused
note: argument gr unused
note: argument H unused



I now reset the outcomes. We now reset globals to hold all the variables again:

In [25]:
%%stata -s gl

global xlist $contvars $dumvars $dumregs
global yvar expoper

Next, I pull the variables into `Mata` and recalculate $\lambda$. Note that because there are multiple treatments, the calculation is a little different than before.

In [26]:
%%stata -s gl
mata:
    st_view(X=.,.,"$xlist")
    st_view(y=.,.,"$yvar")
    w = J(rows(y),1,1)
    
    lambda = 2*max(abs(X))*sqrt(2)/sqrt(rows(X))*(1+ln(rows(X))^(3/2+1)/sqrt(2))^(1/2)   
    printf("Lambda is")
    lambda
    g = J(1,2,1)#I(cols(X))
end


Lambda is
  3.23185689



## Optimization

As before, we first run things with Nelder-Mead, and then let it run with usual optimization routines:

In [27]:
%%stata -s gl -os
mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gltril_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_eq_indepvars(M, 2, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_conv_maxiter(M, 10000)
    moptimize_init_trace_value(M, "off")
    moptimize_init_technique(M, "nm")
    moptimize_init_nmsimplexdeltas(M,J(1,cols(X),.1))
    moptimize(M)

    b = moptimize_result_coefs(M)
end  

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gltril_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_eq_indepvars(M, 2, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_conv_maxiter(M, 500)
    moptimize_init_trace_value(M, "off")
    moptimize_init_eq_coefs(M,1,b[1::cols(b)/2])
    moptimize_init_eq_coefs(M,2,b[cols(b)/2+1::cols(b)])
    moptimize(M)
    moptimize_result_display(M)
end  


                                                Number of obs     =        498

------------------------------------------------------------------------------
           y |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
eq1          |
         sev |   1.184415   .1526159     7.76   0.000      .885293    1.483536
        sev2 |   6.97e-07   .0007355     0.00   0.999    -.0014409    .0014423
    longname |  -.0999266   .1401354    -0.71   0.476    -.3745869    .1747336
       lncas |  -.0094561   .1352477    -0.07   0.944    -.2745367    .2556245
         td1 |    .081713    .277214     0.29   0.768    -.4616164    .6250423
         td2 |  -4.00e-06   .0017101    -0.00   0.998    -.0033557    .0033477
         td3 |   3.43e-06   .0012884     0.00   0.998    -.0025217    .0025286
         td4 |  -9.53e-07   .0011246    -0.00   0.999    -.0022052    .0022033
         td5 |   1.690124   .396758

One can see that the selected predictors are `sev`, `longname`, `lncas`, `td1`, `td5`, `td6`, `td8`, `td9`, and `td10`. 

** Note: ** In the process of fitting the model, the analysis revealed that the dummy `td6` more or less perfectly predicted membership in one or more treatment classes. This is the dummy for "head wounds," which evidently were almost never treated. Hence, we dropped all of these from the subsequent analysis. In any event, here is our pared-down treatment model, as suggested by the lasso: 

The set of variables are thus reduced from 22 to 9 for the multiple outcomes model. 

In [28]:
%%stata -s gl

mlogit expoper sev longname lncas td1 td5 td6 td8 td9 td10, vce(robust)


Iteration 0:   log pseudolikelihood = -406.58183  
Iteration 1:   log pseudolikelihood = -303.65873  
Iteration 2:   log pseudolikelihood = -287.37341  
Iteration 3:   log pseudolikelihood = -284.77562  
Iteration 4:   log pseudolikelihood =  -284.3115  
Iteration 5:   log pseudolikelihood = -284.20538  
Iteration 6:   log pseudolikelihood = -284.18204  
Iteration 7:   log pseudolikelihood = -284.17621  
Iteration 8:   log pseudolikelihood = -284.17508  
Iteration 9:   log pseudolikelihood = -284.17487  
Iteration 10:  log pseudolikelihood = -284.17485  

Multinomial logistic regression                 Number of obs     =        498
                                                Wald chi2(18)     =    6478.25
                                                Prob > chi2       =     0.0000
Log pseudolikelihood = -284.17485               Pseudo R2         =     0.3011

------------------------------------------------------------------------------
             |               Robust
     

We see that there is a conceptual problem with the above, in that those "very large" negative coefficients effectively mean that wounded soldiers with head wounds and neck, trunk, shoulder wounds never wind up with amputations. So, we will proceed after dropping these variables. 

In [30]:
%%stata -s gl

preserve
drop if td6 | td10 | td7
global xlist sev sev2 longname lncas td1 td2 td3 td4 td5 td8 td9 Ky22 Oh16 In54 Oh42 Il13 Mo29 In49 Mo06
mata:
    st_view(X=.,.,"$xlist")
    st_view(y=.,.,"$yvar")
    st_view(w=.,.,"w")
    
    lambda = 2*max(abs(X))*sqrt(2)/sqrt(rows(X))*(1+ln(rows(X))^(3/2+1)/sqrt(2))^(1/2)   
    printf("Lambda is")
    lambda
    g = J(1,2,1)#I(cols(X))
end

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gltril_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_eq_indepvars(M, 2, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_conv_maxiter(M, 50000)
    moptimize_init_trace_value(M, "off")
    moptimize_init_technique(M, "nm")
    moptimize_init_nmsimplexdeltas(M,J(1,cols(X),.1))
    moptimize(M)

    b = moptimize_result_coefs(M)
end  

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gltril_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_eq_indepvars(M, 2, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_conv_maxiter(M, 500)
    moptimize_init_trace_value(M, "off")
    moptimize_init_eq_coefs(M,1,b[1::cols(b)/2])
    moptimize_init_eq_coefs(M,2,b[cols(b)/2+1::cols(b)])
    moptimize(M)
    moptimize_result_display(M)
end  

already preserved
r(621);

(0 observations deleted)

Lambda is
  3.633265754

                 <istmt>:  3499  M not found
------------------------------------------------------------------------------------------------------------------------------------------
r(3499);

command moptimize_init_evaluatortype is unrecognized
r(199);

command moptimize_init_eq_indepvars is unrecognized
r(199);

command moptimize_init_eq_indepvars is unrecognized
r(199);

command moptimize_init_depvar is unrecognized
r(199);

command moptimize_init_userinfo is unrecognized
r(199);

command moptimize_init_userinfo is unrecognized
r(199);

command moptimize_init_userinfo is unrecognized
r(199);

command moptimize_init_singularHmethod is unrecognized
r(199);

command moptimize_init_conv_maxiter is unrecognized
r(199);

command moptimize_init_trace_value is unrecognized
r(199);

command moptimize_init_technique is unrecognized
r(199);

command moptimize_init_nmsimplexdeltas is unrecognized
r(199);

command mop

In this smaller dataset, we see that `sev`, `longname`, `lncas`, `td5`, `td8`, and `td9` matter. Accordingly, I reestimate the model and then create weights for estimates of the outcome equations:

In [34]:
%%stata -s gl

capture drop p0 p1 p2 w

mlogit expoper sev longname lncas td5 td8 td9

predict p0 p1 p2, p
gen w = 1/p0*(expoper == 0) + 1/p1*(expoper == 1) + 1/p2*(expoper == 2)


Iteration 0:   log likelihood =  -295.9093  
Iteration 1:   log likelihood = -224.41593  
Iteration 2:   log likelihood = -218.65211  
Iteration 3:   log likelihood = -218.56947  
Iteration 4:   log likelihood = -218.56934  
Iteration 5:   log likelihood = -218.56934  

Multinomial logistic regression                 Number of obs     =        335
                                                LR chi2(12)       =     154.68
                                                Prob > chi2       =     0.0000
Log likelihood = -218.56934                     Pseudo R2         =     0.2614

------------------------------------------------------------------------------
     expoper |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
0            |  (base outcome)
-------------+----------------------------------------------------------------
1            |
         sev |   1.416791   .1753769     8.08   0

## Outcome equations

I use the same methods as I did earlier - adding interactions for different treatments, and then grouping these interactions with coefficients. This ensures that the same variables will appear as controls in outcome equations. 

Here is the code that creates globals and interactions for all variables:

In [35]:
%%stata -s gl -os

global xlist sev sev2 td1 td2 td3 td4 td5 td8 td9 Ky22 Oh16 In54 Oh42 Il13 Mo29 In49 Mo06
global yvar outcome

global xoutlist
foreach variable in $xlist {
    gen ia_`variable' = `variable'*(expoper == 1)
    gen ie_`variable' = `variable'*(expoper == 2)
    global xoutlist $xoutlist `variable' ia_`variable' ie_`variable'
}

gen amputation = (expoper == 1)
global xoutlist $xoutlist amputation

## Mata functions and implementation

The next a new version of the mata function to lasso outcomes across the two treatments (and one non-treatment). The idea is to get predictor variables that are useful in predicting the incidence of surgery across all three outcome equations. 

In [36]:
%%stata -s gl
mata:
    void gll_aemata(M, todo, b, f, gr, H) {
        
        y  = moptimize_util_depvar(M, 1)
        xb = moptimize_util_xb(M, b, 1)
        
        lam = moptimize_util_userinfo(M, 1)
        g   = moptimize_util_userinfo(M, 2)
        w   = moptimize_util_userinfo(M, 3)
        w   = w/colsum(w)*rows(w)
        
        bt  = b[1::cols(b) - 2]
        gb  = sqrt(rowsum(g)):*sqrt(rowsum((bt:^2):*g))
        norm = colsum(gb)       
       
        lnf  = w:*((y :== 1) :*xb - ln(1 :+ exp(xb)))       
        
        f = sum(lnf) - norm*lam
    }
end   

mata:
    st_view(X=.,.,"$xoutlist")
    st_view(y=.,.,"$yvar")
    st_view(w=.,.,"w")
    
    lambda = 2*max(abs(X))*sqrt(2)/sqrt(rows(X))*(1+ln(rows(X))^(3/2+1)/sqrt(2))^(1/2) 

    g = I(cols(X)/3)#J(1,3,1)
end

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gll_aemata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_trace_value(M, "off")
    moptimize_init_conv_maxiter(M, 600)
    moptimize(M)
    moptimize_result_display(M)
end


note: argument todo unused
note: argument gr unused
note: argument H unused

convergence not achieved

                                                Number of obs     =        335

------------------------------------------------------------------------------
           y |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         sev |  -.4457374   .1589992    -2.80   0.005    -.7573702   -.1341046
      ia_sev |   .0079837   .2190762     0.04   0.971    -.4213978    .4373653
      ie_sev |   .0610567   .1981181     0.31   0.758    -.3272477    .4493611
        sev2 |  -.0000117   .0021237    -0.01   0.996    -.0041741    .0041508
     ia_sev2 |  -8.88e-07   .0013854    -0.00   0.999    -.0027162    .0027145
     ie_sev2 |   8.62e-07   .0013807     0.00   1.000    -.0027052    .0027069
         td1 |   3.78e-06   .0014163     0.00   0.998    -.0027722    .0027798
      ia_td1 |   1.15e-06 

## Estimating treatment effects

From the above, a parsimonious model of outcomes in the case includes: just `sev` and `td5`. In this case, 26 variables have been reduced to 2. 

Therefore, the lasso suggests the following parsimonious model of outcomes and selection into treatments (once again fit with `Stata`'s canned `teffects` package. When actually fitting the models, I will not do it this way because it is not clear what is happening under the hood. But, as a way of previewing results:

In [37]:
%%stata -s gl -os

teffects aipw (outcome sev td5, logit) (expoper sev longname lncas td4 td5 td8 td9, mlogit ), aequations

restore


Iteration 0:   EE criterion =  .00094578  
Iteration 1:   EE criterion =  3.072e-06  
Iteration 2:   EE criterion =  2.466e-06  
Iteration 3:   EE criterion =  3.443e-08  
Iteration 4:   EE criterion =  2.358e-10  

Treatment-effects estimation                    Number of obs     =        335
Estimator      : augmented IPW
Outcome model  : logit by ML
Treatment model: (multinomial) logit
------------------------------------------------------------------------------
             |               Robust
     outcome |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
ATE          |
     expoper |
   (1 vs 0)  |   .0014965   .0781815     0.02   0.985    -.1517365    .1547295
   (2 vs 0)  |   .0894258   .0526902     1.70   0.090    -.0138451    .1926967
-------------+----------------------------------------------------------------
POmean       |
     expoper |
          0  |   .8098308   .0355713 

## Summary of Results

Some comments: 

1. Severity matters a lot in selection into treatment, and it matters alot for outcomes when not treated. But it looks as though amputation and resection/excision more or less mitigate the severity of the wound. 

2. The more complex procedure resection/excision seems to be strongly influenced by the battlefield setting. 

3. It could be argued that amputation seems to be a stop gap measure that works, but isn't as effective as more invasive treatments. The deal with amputation might also be that it was for wounds of the extremities, which have a high survival rate anyways. So, the change in the survival rate is estimated to be smaller. 

# One last experiment

Look at wounds that can be judged to be more critical, and fit our original model to them. So, what we will do is run a simple treatment and outcome Lasso on wounds to the head; hip; knee; leg; neck, trunk, or shoulder; and thigh. In most cases, these operations involved quite a bit more work and intricacy to get this to work, is the argument.

So, we have the following:

In [38]:
%%stata -s gl

gen critloc = td6 | td7 | td8 | td9 | td10 | td11
tab critloc


    critloc |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        171       34.34       34.34
          1 |        327       65.66      100.00
------------+-----------------------------------
      Total |        498      100.00



In [41]:
%%stata -s gl
global xlist sev sev2 lncas longname td6 td7 td8 td9 td10 Ky22 Oh16 In54 Oh42 Il13 Mo29 In49 Mo06
global yvar operated
preserve
keep if critloc
mata:
    st_view(X=.,.,"$xlist")
    st_view(y=.,.,"$yvar")
    w = J(rows(y),1,1)
    
    lambda = 2*max(abs(X))/sqrt(rows(X))*(1+ln(rows(X))^(3/2+1))^(1/2)
    printf("Lambda is")
    lambda
    g = I(cols(X))
    printf("Number variables")
    cols(X)
end



already preserved
r(621);

(0 observations deleted)

Lambda is
  3.068787788

Number variables
  17



## Running the Treatment Lasso

In [42]:
%%stata -s gl 
mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gll_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_trace_value(M, "off")
    moptimize_init_conv_maxiter(M, 5000)
    moptimize(M)
    moptimize_result_display(M)
end 


                                                Number of obs     =        327

------------------------------------------------------------------------------
           y |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         sev |   .8176354   .1545953     5.29   0.000     .5146341    1.120637
        sev2 |   2.15e-06   .0015731     0.00   0.999     -.003081    .0030853
       lncas |  -.4558087   .1352639    -3.37   0.001     -.720921   -.1906963
    longname |   .0000125    .004524     0.00   0.998    -.0088544    .0088793
         td6 |  -.5339225   .5361905    -1.00   0.319    -1.584837    .5169916
         td7 |  -3.83e-08   .0003125    -0.00   1.000    -.0006125    .0006124
         td8 |    .552261    .552753     1.00   0.318    -.5311149    1.635637
         td9 |  -9.77e-08   .0003868    -0.00   1.000    -.0007582     .000758
        td10 |  -8.17e-06   .0029865    -0.00   0.

Here, the variables not zeroed out be the lasso are `sev`, `lncas`, `td6`, `td8` and `In49`. One thing that is interesting is that the `longname` variable is zeroed. Maybe this is because soldiers wounded in these locations were in no position to argue!

Now, let's estimate the outcome model, after dropping all the interactions we had before and remaking them:

In [44]:
%%stata -s gl

logit $yvar sev lncas td6 td8 In49

capture drop ps
capture drop w
predict ps, p

gen w = operated*1/ps + (1-operated)*1/(1-ps)


Iteration 0:   log likelihood = -176.08924  
Iteration 1:   log likelihood =  -140.0948  
Iteration 2:   log likelihood = -137.54747  
Iteration 3:   log likelihood = -137.50961  
Iteration 4:   log likelihood =  -137.5095  
Iteration 5:   log likelihood =  -137.5095  

Logistic regression                             Number of obs     =        327
                                                LR chi2(5)        =      77.16
                                                Prob > chi2       =     0.0000
Log likelihood =  -137.5095                     Pseudo R2         =     0.2191

------------------------------------------------------------------------------
    operated |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         sev |   .8897476   .1607862     5.53   0.000     .5746124    1.204883
       lncas |  -.5385554   .1420631    -3.79   0.000    -.8169939   -.2601169
         td6 |  

In [45]:
%%stata -s gl
capture drop io*

global xoutlist
foreach variable in $xlist {
    gen io_`variable' = `variable'*operated
    global xoutlist $xoutlist `variable' io_`variable'
}

global yvar outcome

In [46]:
%%stata -s gl 
mata:
    st_view(X=.,.,"$xoutlist")
    st_view(y=.,.,"$yvar")
    st_view(w=.,.,"w")
    
    lambda = 2*max(abs(X))/sqrt(rows(X))*(1+ln(rows(X))^(3/2+1))^(1/2)
    printf("Lambda is")
    lambda
    g = I(cols(X)/2)#J(1,2,1)
end


Lambda is
  3.068787788



In [47]:
%%stata -s gl -os

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gll_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_conv_maxiter(M, 50000)
    moptimize_init_trace_value(M, "off")
    moptimize_init_technique(M, "nm")
    moptimize_init_nmsimplexdeltas(M,J(1,cols(X),.05))
    moptimize(M)
    b = moptimize_result_coefs(M)
end  

mata:
    M = moptimize_init()
    moptimize_init_evaluator(M, &gll_mata())
    moptimize_init_evaluatortype(M, "d0")
    moptimize_init_eq_indepvars(M, 1, X)
    moptimize_init_depvar(M, 1, y)
    moptimize_init_userinfo(M, 1, lambda)
    moptimize_init_userinfo(M, 2, g)
    moptimize_init_userinfo(M, 3, w)
    moptimize_init_singularHmethod(M, "hybrid")
    moptimize_init_conv_maxiter(M, 1000)
    moptimize_init_trace_value(M, "off") 
    moptimize_init_eq_coefs(M, 1, b)    
    moptimize(M)
    moptimize_result_display(M)
end


                                                Number of obs     =        327

------------------------------------------------------------------------------
           y |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         sev |  -1.014939   .1980172    -5.13   0.000    -1.403045   -.6268322
      io_sev |   .5234818   .2756887     1.90   0.058    -.0168581    1.063822
        sev2 |  -3.65e-06   .0013575    -0.00   0.998    -.0026642    .0026569
     io_sev2 |   9.93e-07   .0010051     0.00   0.999    -.0019689    .0019709
       lncas |   .0150281    .114885     0.13   0.896    -.2101423    .2401985
    io_lncas |   .1076849     .20834     0.52   0.605     -.300654    .5160237
    longname |  -4.96e-07   .0006218    -0.00   0.999    -.0012192    .0012182
 io_longname |   1.11e-06   .0007265     0.00   0.999    -.0014228     .001425
         td6 |  -2.73e-07   .0007733    -0.00   1.

In [48]:
%%stata -s gl

teffects aipw (outcome sev lncas td8 td9 td10 Oh16, logit) (operated sev lncas td6 td8 In49, logit), aequations


Iteration 0:   EE criterion =  .00093233  
Iteration 1:   EE criterion =  3.724e-06  
Iteration 2:   EE criterion =  2.682e-06  
Iteration 3:   EE criterion =  7.865e-08  
Iteration 4:   EE criterion =  1.318e-10  

Treatment-effects estimation                    Number of obs     =        327
Estimator      : augmented IPW
Outcome model  : logit by ML
Treatment model: logit
------------------------------------------------------------------------------
             |               Robust
     outcome |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
ATE          |
    operated |
   (1 vs 0)  |   .1744163   .0466182     3.74   0.000     .0830464    .2657863
-------------+----------------------------------------------------------------
POmean       |
    operated |
          0  |   .6800866   .0315865    21.53   0.000     .6181781    .7419951
-------------+-------------------------------------

# On to the Next Workbook

The primary purpose of this workbook was to pare down the models with the lasso. Please see [this workbook](Estimation Tables and Figures for Presentation.ipynb) for the next step, which involves formally estimating models and treatment effects, and computing standard errors.  

For the time being, I am going to leave the above analysis out of the paper, but I will mention it. 